In [1]:
import os

import dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

dotenv.load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Reset and fill Vector Database

In [2]:
# Parameters
fp_data = "./data/dutch-news-articles.csv"

# Splitting and data size
chunk_size = 1_000
chunk_overlap = 200
N_docs = 100  # Set to None to process all documents

# Embedding model for retrieval
# embed_id = "sentence-transformers/all-mpnet-base-v2"
# embed_id = "sentence-transformers/all-MiniLM-L6-v2"  # Faster model
embed_id = "paraphrase-multilingual-MiniLM-L12-v2"  # Multilingual model, including Dutch
embed_model_kwargs = {"device": "cpu"}
embed_encode_kwargs = {"normalize_embeddings": False}


In [3]:
# Load data from CSV, set metadata
loader = CSVLoader(file_path=fp_data, encoding="utf-8", source_column="url", metadata_columns=["datetime", "category", "url"])
docs = loader.load()

# Preview the first document
print(docs[0].page_content[:100])
print(docs[0].page_content[-100:])

title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavon
jvoorbeeld gas uit Rusland. De kerncentrale leverde bijna driekwart van de Litouwse energiebehoefte.


In [4]:
# Chunking the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=True)
splits = text_splitter.split_documents(docs[:N_docs] if N_docs else docs)

print(f"{len(splits)} chuncks in total")

220 splits in total


In [5]:
print(f"Metadata: {splits[0].metadata}")
print(f"Contents:\n\n{splits[0].page_content}")

Metadata: {'source': 'https://nos.nl/artikel/126231-enige-litouwse-kerncentrale-dicht.html', 'row': 0, 'datetime': '2010-01-01 00:49:00', 'category': 'Buitenland', 'url': 'https://nos.nl/artikel/126231-enige-litouwse-kerncentrale-dicht.html', 'start_index': 0}
Contents:

title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen beloofde al in 2004 om de centrale te sluiten in ruil voor toetreding tot de Europese Unie. De EU wilde sluiting omdat de kerncentrale bij de stad Visiginas mogelijk niet veilig was. Nucleaire ramp De centrale is een grotere versie van die bij Tsjernobyl. Die ontplofte in 1986 en veroorzaakte een nucleaire wolk die over een groot deel van Europa trok. Dat was de grootste nucleaire ramp in de geschiedenis. Voor Litouwen betekent de sluiting dat het land een goedkope bron van energie kwijt is. Het wordt nu veel afhankelijker v

In [7]:
# Load embedding model for retrieval
embed_model = HuggingFaceEmbeddings(model_name=embed_id, model_kwargs=embed_model_kwargs, encode_kwargs=embed_encode_kwargs)

# Create the vectorstore, save to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=embed_model, persist_directory="./chroma_db")  # Investigate persist_directory to store the vectorstore

## Example

In [9]:
# Parameters
vectorstore_search_kwargs = {"k": 6}
question = "Welke kerncentrale is recent buiten bedrijf gesteld?"

In [8]:
# Convert vectorstore to retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs=vectorstore_search_kwargs)

In [11]:
# Retrieve documents
retrieved_docs = retriever.invoke(question)

print(f"{len(retrieved_docs)} retrieved docs")
print("Contents:")
for i, doc in enumerate(retrieved_docs):
    print(f"Doc {i}:\t{doc.page_content}")

6 retrieved docs
Contents:
Doc 0: title: Enige Litouwse kerncentrale dicht
content: De enige kerncentrale van Litouwen is oudjaarsavond om 23.00 uur buiten gebruik gesteld. Dat verliep zonder problemen, aldus de directeur. Litouwen beloofde al in 2004 om de centrale te sluiten in ruil voor toetreding tot de Europese Unie. De EU wilde sluiting omdat de kerncentrale bij de stad Visiginas mogelijk niet veilig was. Nucleaire ramp De centrale is een grotere versie van die bij Tsjernobyl. Die ontplofte in 1986 en veroorzaakte een nucleaire wolk die over een groot deel van Europa trok. Dat was de grootste nucleaire ramp in de geschiedenis. Voor Litouwen betekent de sluiting dat het land een goedkope bron van energie kwijt is. Het wordt nu veel afhankelijker van bijvoorbeeld gas uit Rusland. De kerncentrale leverde bijna driekwart van de Litouwse energiebehoefte.
Doc 1: Olympische organisatie, heeft 650 bedrijven gevraagd om het vervoer van werknemers terug te dringen, maar daarop is nog niet 